# Agent
https://python.langchain.com/v0.1/docs/use_cases/tool_use/quickstart/#chains

## Import

In [156]:
from dotenv import load_dotenv
import os
from pathlib import Path

from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

from groq import Groq
import base64
import json

## Setup

In [93]:
# 刪除環境變量
if "GROQ_API_KEY" in os.environ:
    del os.environ["GROQ_API_KEY"]

In [94]:
# 找根目錄
def find_project_root(current_path, marker=".git"):
    current_path = Path(current_path).resolve()
    for parent in current_path.parents:
        if (parent / marker).exists():
            return parent
    return None

current_path = os.getcwd()
project_root = find_project_root(current_path, marker=".git")
print("Project root:", project_root)

# Load .env file
print(f"Successfully loaded env variables: {load_dotenv(project_root / ".env")}")

Project root: /Users/allen/Documents/code/Exchange_QA_Chatbot
Successfully loaded env variables: True


In [95]:
# Load env variables into python variables
print("Loaded env variables:")
print(f"GROQ_API_KEY = {os.getenv("GROQ_API_KEY")}")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

Loaded env variables:
GROQ_API_KEY = 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


## LLM

### Tools

In [225]:
@tool
def image_interpreting(image_path: str) -> str:
    """Interpret, explain, analyse or describe the content and underlying logic of any given image and return in JSON format.

    Args:
        image_path: the image file path
    """
    # Function to encode the image
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    
    base64_image = encode_image(image_path)
    request_text = "You are an assistant skilled at interpreting the content and underlying logic in images. Texts in this image are in Traditional Chinese. List what you observe in this image in JSON format."
    client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
    chat_completion = client.chat.completions.create(
        model="llama-3.2-90b-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {   
                        "type": "text",
                        "text": request_text
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    },
                ],
            }
        ],
        temperature=0.0,
        top_p=1,
        stream=False,
        response_format={"type": "json_object"},
        stop=None,
    )
    return str(json.loads(chat_completion.choices[0].message.content))
    # return "{'title': 'Step2. 校外徵選', 'sections': [{'title': 'Nomination', 'description': '校方提名', 'dates': ['第一學期：2月至3月', '第二學期：8月至9月']}, {'title': 'Application', 'description': '學生申請文件', 'description2': '申請文件及期程依各交換校規定辦理'}, {'title': 'Result', 'description': '結果通知', 'description2': '母校通知', 'description3': '交換校通知'}, {'title': '學生負責事項', 'description': '包含住宿、簽證、交通、保險、健康檢查，限制提領帳戶...等(依各交換校及所在國家規定)'}], 'button': {'text': '看詳細資訊', 'color': 'orange'}}"

@tool
def document_retrieval(string: str) -> str:
    """A powerful tool designed to retrieve relevant information from the robust document database using natural language questions or key phrases.

    Args:
        string: a sentence or a key phrase to retrieve relevant information from the database
    """
    text = f"""
1. 繳交文件
2. 聯絡交換學校
"""
    return text

tools = [image_interpreting, document_retrieval]

### Check tools

In [205]:
print(f"Name:\n{image_interpreting.name}")
print("=====")
print(f"Description:\n{image_interpreting.description}")
print("=====")
print(f"Args:\n{image_interpreting.args}")

Name:
image_interpreting
=====
Description:
Interpret, explain or analyse the content and underlying logic of any given image and return in JSON format.

    Args:
        image_path: the image file path
=====
Args:
{'image_path': {'title': 'Image Path', 'type': 'string'}}


In [206]:
image_interpreting.invoke({"image_path": "../data/images/Step2校外徵選.jpg"})

"{'title': 'Step2. 校外徵選', 'sections': [{'title': 'Nomination', 'description': '校方提名', 'dates': ['第一學期：2月至3月', '第二學期：8月至9月']}, {'title': 'Application', 'description': '學生申請文件', 'description2': '申請文件及期程依各交換校規定辦理'}, {'title': 'Result', 'description': '結果通知', 'description2': '母校通知', 'description3': '交換校通知'}, {'title': '學生負責事項', 'description': '包含住宿、簽證、交通、保險、健康檢查，限制提領帳戶...等(依各交換校及所在國家規定)'}], 'button': {'text': '看詳細資訊', 'color': 'orange'}}"

### Agent

In [226]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.7,
    max_retries=2,
    api_key=GROQ_API_KEY
)

In [227]:
# Get the prompt to use - can be replaced with any prompt that includes variables "agent_scratchpad" and "input"!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

/Users/allen/Documents/code/Exchange_QA_Chatbot/venv/lib/python3.12/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [228]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [229]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [232]:
# user_prompt = "'../data/images/Step2校外徵選.jpg' 請問在校方提名之後，學生要做什麼？"
# user_prompt = "'../data/images/Step2校外徵選.jpg' 請問校方提名的第一學期是幾月至幾月？"
user_prompt = f"""
'../data/images/Step2校外徵選.jpg'
Recommend using tool "image_interpreting" to interpret, explain or analyse the content and underlying logic of any given image and return in JSON format.
If information in the image isn't informative enough to answer the Human's message below, you could use the powerful tool "document_retrieval" to retrieve relevant information from the robust document database using 'Human's message' below.

Human's message:
"學生負責事項有哪些？"
"""


agent_executor.invoke(
    {
        "input": user_prompt
    }
)



> Entering new AgentExecutor chain...

Invoking: `image_interpreting` with `{'image_path': '../data/images/Step2校外徵選.jpg'}`


{'title': 'Step2. 校外徵選', 'sections': [{'title': 'Nomination', 'description': '校方提名', 'dates': ['第一學期：2月至3月', '第二學期：8月至9月']}, {'title': 'Application', 'description': '學生申請文件', 'description2': '申請文件及期程依各交換校規定辦理'}, {'title': 'Result', 'description': '結果通知', 'description2': '母校通知', 'description3': '交換校通知'}, {'title': '學生負責事項', 'description': '包含住宿、簽證、交通、保險、健康檢查，限制提領帳戶...等(依各交換校及所在國家規定)'}], 'button': {'text': '看詳細資訊', 'color': 'orange'}}
Invoking: `document_retrieval` with `{'string': '學生負責事項'}`



1. 繳交文件
2. 聯絡交換學校
學生負責事項包含繳交文件、聯絡交換學校、住宿、簽證、交通、保險、健康檢查等，具體事項依各交換校及所在國家規定而定。

> Finished chain.


{'input': '\n\'../data/images/Step2校外徵選.jpg\'\nRecommend using tool "image_interpreting" to interpret, explain or analyse the content and underlying logic of any given image and return in JSON format.\nIf information in the image isn\'t informative enough to answer the Human\'s message below, you could use the powerful tool "document_retrieval" to retrieve relevant information from the robust document database using \'Human\'s message\' below.\n\nHuman\'s message:\n"學生負責事項有哪些？"\n',
 'output': '學生負責事項包含繳交文件、聯絡交換學校、住宿、簽證、交通、保險、健康檢查等，具體事項依各交換校及所在國家規定而定。'}

### Ignore

In [ ]:
system_prompt = f"""
When you receive a tool call response, use the output to format an answer to the orginal user question.

You are a helpful assistant with tool calling capabilities.
"""

user_prompt = f"""
Here's a list of images that you can perform Optical Character Recognition (OCR) on to extract the texts in the image.
The images are provided with their file path:
1. "data/images/Step1校內徵選.jpg"

Step1校內徵選的流程是什麼？
"""

# I would like to know what texts are written on "Step1 校內徵選" image. Please help me out.

# I provided an image file with the file path: "data/images/Step1校內徵選.jpg".
# Please perform Optical Character Recognition (OCR) on the image and show me the result.

messages = [
    ("system", system_prompt),
    ("human", user_prompt),
]

In [ ]:
# Invoke
response = llm_with_tools.invoke(messages)

for key, value in vars(response).items():
    print(f"{key}: {value}")

content: 
additional_kwargs: {'tool_calls': [{'id': 'call_fjk0', 'function': {'arguments': '{"image_file_path": "data/images/Step1校內徵選.jpg"}', 'name': 'image_ocr'}, 'type': 'function'}]}
response_metadata: {'token_usage': {'completion_tokens': 27, 'prompt_tokens': 356, 'total_tokens': 383, 'completion_time': 0.098181818, 'prompt_time': 0.042432677, 'queue_time': 0.019514951999999995, 'total_time': 0.140614495}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_fcc3b74982', 'finish_reason': 'tool_calls', 'logprobs': None}
type: ai
name: None
id: run-d10394e2-827a-477d-bd05-5b595067b156-0
example: False
tool_calls: [{'name': 'image_ocr', 'args': {'image_file_path': 'data/images/Step1校內徵選.jpg'}, 'id': 'call_fjk0', 'type': 'tool_call'}]
invalid_tool_calls: []
usage_metadata: {'input_tokens': 356, 'output_tokens': 27, 'total_tokens': 383}
